In [ ]:
from django.core.exceptions import ObjectDoesNotExist
from tqdm import tqdm
from dumper.utils import gsheet_to_df
from apis_core.utils import get_object_from_pk_or_uri

In [ ]:
df = gsheet_to_df("1-_WXbdmpLzz_9vBiZ1y9v2tle_26m4tAhPTGzkkin5E")

In [ ]:
df

In [ ]:
relation_type_lookup = {
    "1049": {
        "relation_type":PersonWork,
        "relation": PersonWorkRelation.objects.get(id=1049),
        "target_class": Work
    },
    "1052": {
            "relation_type": PersonWork,
            "relation": PersonWorkRelation.objects.get(id=1052),
            "target_class": Work
        },
    "1053": {
            "relation_type": PersonWork,
            "relation": PersonWorkRelation.objects.get(id=1053),
            "target_class": Work
        },
    "1181": {
            "relation_type": PersonPlace,
            "relation": PersonPlaceRelation.objects.get(id=1181),
            "target_class": Place
        },
    "1217": {
            "relation_type": PersonWork,
            "relation": PersonWorkRelation.objects.get(id=1217),
            "target_class": Work
        }
}

In [ ]:
does_not_exist = set()
for g, ndf in df.groupby("relation-id"):
    relation = relation_type_lookup[str(g)]["relation"]
    relation_class = relation_type_lookup[str(g)]["relation_type"]
    target_class = relation_type_lookup[str(g)]["target_class"]
    print(relation, relation_class, target_class)
    if "ork" in relation_class.__name__:
            print("work")
    else:
        print("place")
    for i, row in tqdm(ndf.iterrows(), total=len(ndf)):
        try:
            source = Person.objects.get(pk=row["source_id"])
        except ObjectDoesNotExist:
            does_not_exist.add(row["source_id"])
            continue
        try:
            target = target_class.objects.get(pk=row["target_id"])
        except ObjectDoesNotExist:
            does_not_exist.add(row["target_id"])
        if isinstance(row["relation_start_date_written"], str):
            start_date_written = row["relation_start_date_written"]
            end_date_written = row["relation_end_date_written"]
        else:
            start_date_written = None
            end_date_written = None
        if "ork" in relation_class.__name__:
               relation_class.objects.get_or_create(
                   related_person=source,
                   related_work=target,
                   relation_type=relation,
                   start_date_written=start_date_written,
                   end_date_written=end_date_written
               )
        else:
            relation_class.objects.get_or_create(
                   related_person=source,
                   related_place=target,
                   relation_type=relation,
                   start_date_written=start_date_written,
                   end_date_written=end_date_written
               )
